# Group Chat Orchestration Pattern with Semantic Kernel

This notebook demonstrates the Group Chat Orchestration pattern using Microsoft Semantic Kernel in Python. Group chat orchestration involves multiple agents collaborating simultaneously on a shared task, leveraging Azure OpenAI services for intelligent responses.

### Key Features:
- **Collaborative Execution**: Agents work together in real-time to address a shared task.
- **Task Specialization**: Each agent contributes based on its expertise.
- **Dynamic Interaction**: Agents can respond to each other's outputs, creating a dynamic and interactive workflow.

### Example:
**Scientific Discussion**: Multiple agents (PhysicsExpert, ChemistryExpert, and BiologyExpert) collaborate to answer the question "What is the impact of temperature on living organisms?".

The notebook utilizes `GroupChatOrchestration` and Semantic Kernel's agent orchestration framework to manage and execute tasks collaboratively, showcasing the power and flexibility of Semantic Kernel.

In [ ]:
# Import required libraries
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents.orchestration.group_chat import GroupChatOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime

# Azure OpenAI configuration
# Ensure api_key and endpoint are strings
api_key = "YOUR_AZURE_OPENAI_API_KEY"
endpoint = "YOUR_AZURE_OPENAI_ENDPOINT"
deployment_name = "YOUR_DEPLOYMENT_NAME"

# Initialize Azure OpenAI service
openai_service = AzureChatCompletion(api_key=api_key, endpoint=endpoint, deployment_name=deployment_name)

**Expected Output**: Package installation messages, successful imports, and Azure OpenAI service configuration confirmation.

In [2]:
# Define agents with descriptions
physics_agent = ChatCompletionAgent(
    name="PhysicsExpert",
    description="An expert in physics, focusing on the impact of temperature on physical systems.",
    instructions="You are a physics expert. Explain the impact of temperature on physical systems, such as the behavior of gases and solids.",
    service=openai_service,
)

chemistry_agent = ChatCompletionAgent(
    name="ChemistryExpert",
    description="An expert in chemistry, specializing in temperature effects on chemical reactions.",
    instructions="You are a chemistry expert. Discuss how temperature affects chemical reactions and molecular stability.",
    service=openai_service,
)

biology_agent = ChatCompletionAgent(
    name="BiologyExpert",
    description="An expert in biology, analyzing temperature effects on living organisms.",
    instructions="You are a biology expert. Describe the effects of temperature on living organisms, including metabolism and survival.",
    service=openai_service,
)

agents = [physics_agent, chemistry_agent, biology_agent]

**Expected Output**: Confirmation messages showing that the scientific expert agents (Physics, Chemistry, Biology) have been created and the group chat orchestration is configured.

In [ ]:
# Set up group chat orchestration - Following Microsoft documentation
from semantic_kernel.agents import RoundRobinGroupChatManager
from semantic_kernel.contents import ChatMessageContent
import asyncio
import traceback

# Define agent response callback to capture responses
captured_responses = []

def agent_response_callback(message: ChatMessageContent) -> None:
    """Callback to capture each agent's response as the conversation progresses"""
    # Get full content but limit to reasonable length for readability
    content = str(message.content) if message.content else str(message)
    
    # Only truncate if extremely long (allow full sentences)
    max_length = 500  # Increased from 200 to show more complete responses
    if len(content) > max_length:
        # Try to truncate at sentence boundary
        truncate_pos = content.rfind('.', 0, max_length)
        if truncate_pos > max_length // 2:  # If we found a reasonable sentence break
            content = content[:truncate_pos + 1] + "..."
        else:
            content = content[:max_length] + "..."
    
    # Avoid duplicates by checking if this exact response was already captured
    response_key = (message.name, content)
    if response_key not in [(name, resp) for name, resp in captured_responses]:
        captured_responses.append((message.name, content))
        print(f"{message.name}: {content}")
        print()

print(f"Number of agents configured: {len(agents)}")
for agent in agents:
    print(f"  - {agent.name}: {agent.description}")

# Create group chat orchestration with proper callback
manager = RoundRobinGroupChatManager(max_rounds=3)  # 3 rounds to let each agent respond
group_chat_orchestration = GroupChatOrchestration(
    members=agents,
    manager=manager,
    agent_response_callback=agent_response_callback,
)

# Start runtime
runtime = InProcessRuntime()
runtime.start()

# Execute group chat orchestration
try:
    print("\nStarting group chat orchestration...")
    print("***** Group Chat Results *****")
    print()
    
    # Clear previous responses
    captured_responses.clear()
    
    orchestration_result = await group_chat_orchestration.invoke(
        task="What is the impact of temperature on living organisms? Each expert should provide a brief response from their domain perspective.",
        runtime=runtime,
    )

    # Wait for orchestration to complete
    print("Waiting for orchestration to complete...")
    final_result = await orchestration_result.get(timeout=300)
    
    print(f"\n***** Discussion Complete *****")
    print(f"Total responses captured: {len(captured_responses)}")
    
    # Only show final result if it's meaningfully different from what we already captured
    if final_result and hasattr(final_result, 'content'):
        final_content = str(final_result.content)
        # Check if this final result is different from any of the captured responses
        is_duplicate = any(final_content in resp_content for _, resp_content in captured_responses)
        if not is_duplicate and len(final_content.strip()) > 50:
            # Show a brief summary if it's genuinely different
            if len(final_content) > 300:
                final_content = final_content[:300] + "..."
            print(f"\n***** Final Summary *****")
            print(f"Orchestration Summary: {final_content}")
    
except asyncio.TimeoutError:
    print("The orchestration timed out. Consider increasing the timeout or debugging agent execution.")
except Exception as e:
    print("An error occurred during orchestration:")
    print(traceback.format_exc())

# Stop runtime
await runtime.stop_when_idle()

Number of agents configured: 3
  - PhysicsExpert: An expert in physics, focusing on the impact of temperature on physical systems.
  - ChemistryExpert: An expert in chemistry, specializing in temperature effects on chemical reactions.
  - BiologyExpert: An expert in biology, analyzing temperature effects on living organisms.

Starting group chat orchestration...
***** Group Chat Results *****

Waiting for orchestration to complete...
🔬 PhysicsExpert: From a physics perspective, temperature significantly impacts living organisms by influencing the physical and chemical processes within their bodies. At the molecular level, temperature affects the kinetic energy of particles, which in turn impacts reaction rates, diffusion, and the physical state of molecules such as proteins and lipids.

For example:

1. **Enzyme Activity**: Enzymes, which are critical for metabolism, have an optimal temperature range....

🔬 ChemistryExpert: Certainly! Adopting the persona of ChemistryExpert now.

From 

**Expected Output**: A proper group chat orchestration with complete agent responses (up to 500 characters or sentence boundaries):



